In [1]:
#!pip uninstall transformers torch torchvision numpy -y

In [2]:
!pip install transformers torch torchvision numpy --upgrade

  Using cached torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached torchvision-0.22.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
Using cached torch-2.7.0-cp311-cp311-manylinux_2_28_x86_64.whl (865.2 MB)
Using cached torchvision-0.22.0-cp311-cp311-manylinux_2_28_x86_64.whl (7.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch-2.6.0+cu124
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled torchvision-0.21.0+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but yo

In [3]:
# Show versions:
!pip freeze | grep tensorflow
!pip freeze | grep torch
!pip freeze | grep numpy
# Show Python version
import os
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

tensorflow==2.18.0
tensorflow-datasets==4.9.8
tensorflow-hub==0.16.1
tensorflow-io-gcs-filesystem==0.37.1
tensorflow-metadata==1.17.1
tensorflow-probability==0.25.0
tensorflow-text==2.18.1
tensorflow_decision_forests==1.11.0
torch==2.7.0
torchao==0.10.0
torchaudio @ https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision==0.22.0
numpy==2.2.6
/usr/bin/python3
3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
sys.version_info(major=3, minor=11, micro=12, releaselevel='final', serial=0)


In [5]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Check PyTorch version
print(torch.__version__)
print(torch.cuda.is_available)

# torch.set_default_device(device='cuda:0')
# torch.get_default_device()

# print(torch.accelerator.current_accelerator().type)

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Define the text
texts = [
  "Aeronautics and Space Reports to the Congress, 1958-1984. Bethesda, Md. : University Publications of America, 2012. Astronautics United States History Sources. Manned space flight History Sources. Outer space Exploration United States. United States. National Aeronautics and Space Administration.",
  "Records of President Johnson's Commission on Law Enforcement, Part 1 : Commission Correspondence and Memoranda. Bethesda, Md. : University Publications of America, 2012. Law enforcement United States History 20th century Sources. Crime United States History 20th century Sources. United States. President's Commission on Law Enforcement and Administration of Justice.",
  "Associated Press, Wire Copy on the Assassinations of President John F. Kennedy and Senator Robert F. Kennedy. Bethesda, Md. : University Publications of America, 2012. Associated Press. Kennedy, John F. (John Fitzgerald), 1917-1963 Assassination Sources. Kennedy, Robert F., 1925-1968 Assassination."
]

def get_sentence_embedding(text):
    # Tokenize and prepare input tensors
    inputs = tokenizer(text, return_tensors='pt')
    # Disable gradient calculation
    with torch.no_grad():
        # Get model outputs
        outputs = model(**inputs)
    # Extract the last hidden states (embeddings)
    last_hidden_states = outputs.last_hidden_state
    # Average token embeddings
    sentence_embedding = torch.mean(last_hidden_states, dim=1).numpy()
    return sentence_embedding

# Generate embeddings for texts
embeddings = [get_sentence_embedding(text) for text in texts]

# Query text
query_text = "Commission Correspondence and Memoranda. Bethesda, Md"
query_embedding = get_sentence_embedding(query_text)

# Compute cosine similarities
similarities = cosine_similarity(query_embedding, np.vstack(embeddings))

# Print query text
print (f"Query text: {query_text}")

# Print similarities
for i, text in enumerate(texts):
    print(f"Similarity with '{text}': {similarities[0][i]}")

# # Print the dimensions of the embeddings
# print("Shape of the last hidden state (embeddings):", last_hidden_states.shape)

# # Print embeddings for each token along with their vector dimension
# tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
# for token, embedding in zip(tokens, last_hidden_states[0]):
#     print(f"Token: {token}, Embedding Dimension: {embedding.shape}, Embedding (first 5 components): {embedding[:5]}...")  # Display first 5 components for brevity

2.7.0+cu126
<function is_available at 0x7b48defeafc0>


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Query text: Commission Correspondence and Memoranda. Bethesda, Md
Similarity with 'Aeronautics and Space Reports to the Congress, 1958-1984. Bethesda, Md. : University Publications of America, 2012. Astronautics United States History Sources. Manned space flight History Sources. Outer space Exploration United States. United States. National Aeronautics and Space Administration.': 0.7034063339233398
Similarity with 'Records of President Johnson's Commission on Law Enforcement, Part 1 : Commission Correspondence and Memoranda. Bethesda, Md. : University Publications of America, 2012. Law enforcement United States History 20th century Sources. Crime United States History 20th century Sources. United States. President's Commission on Law Enforcement and Administration of Justice.': 0.7431371212005615
Similarity with 'Associated Press, Wire Copy on the Assassinations of President John F. Kennedy and Senator Robert F. Kennedy. Bethesda, Md. : University Publications of America, 2012. Associa